# 9-ая неделя:

### Контрольные вопросы:
1) Что такое шаблон и какие разновидности шаблонов существуют?
    Фактически, шаблон - это некоторый общий алгоритм, “трафарет” для создания аналогичных функций/классов для разных типов данных. 
    Существуют следующие разновидности:
        а) шаблоны функций — это семейство функций, которые служат образцом для создания других подобных функций.
        б) шаблоны классов (определение аналогично шаблонам функций, только для классов)
        в) шаблоны переменных
        г) шаблоны типов 

2) Каким образом осуществляется двухэтапная трансляция шаблона?
    Сначала происходит проверка кода на общие ошибки (без подстановки конкретного типа). Например, не поставили “;” и т.д.
    (Если на данном этапе не возникает никаких проблем, то переходим к пункту 2)
    Проверка кода с конкретным типом данных.

3) Что предпочитает компилятор при перегрузке шаблона функции?
    Предпочтение отдаётся не шаблонной функции, потому что в случае выбора шаблонной функции придётся выполнять инстанцирование шаблона, которое проходит в два этапа. А компилятор предпочитает пойти по пути “наименьшего сопротивления”.

4) В чем заключается особенность инстанцирования шаблонов классов?
    Инстанцирование выполняется только для тех функций-членов, которые используются, т.е. если мы определили какую-то функцию-член, но не вызвали ей, то инстанцироваться она не будет.

5) Когда необходимы полная или частичная специализации шаблона?
    Иногда нежелательно, чтобы шаблон определял один и тот же код для любого типа данных. Например, нужно проверить какие-то особые случаи для конкретного типа параметров. 
    Полная специализации подходит для случаев, в которых все параметры становятся специализированными. Например, 
        template <>
        class X < int, double > //для класса Х специализировали оба принимаемых параметра 

    Частичная специализация необходима, когда нужно переопределить только часть передаваемых параметров. 
    Например,
        template < typename T1 >
        class X < T1, int >
    ИЛИ:
        template < typename T1, typename T2 >
        class X < T1*, T2* >

    P.S. Стоит также отметить, что частичная специализация применима только для шаблонов классов.

6) Дополнительный вопрос: 
    Для какого типа специализация стандартного шаблона класса std::vector имеет смысл с точки зрения оптимизации потребления памяти и почему?
    Шаблон данного класса включает в себя 2 параметра: value_type и allocator_type. 
    Второй параметр отвечает за динамическое выделение памяти и, как известно, память обычно выделяется с запасом. Поэтому с точки зрения оптимизации потребления памяти имеет смысл специализация стандартного шаблона класса std::vector для типа данных bool, который принимает два фиксированных значения - false (== 0)/true (==1) => Можно отвести 1 бит на хранение “bool”, когда в обычном случае тип bool занимает 8 бит.  
    В стандартной библиотеке как раз реализована такая специализация - std::vector <bool> .

##                                                             Task 1

In [ ]:
//first task .hpp file

#pragma once

template < typename T, typename F >
void merge(T* a, unsigned int size, int left, int right, F f)
{

    const auto m = left + (right - left) / 2;
    int first = left;
    int last = m + 1;


    std::vector <T> temp(size);


    for (int j = left; j <= right; j++)
    {
        if ((first <= m) && ((last > right) || f(a[first], a[last])))
        {
            temp[j] = a[first];
            ++first;
        }
        else
        {
            temp[j] = a[last];
            ++last;
        }
    }

    for (int j = left; j <= right; j++)
    {
        a[j] = temp[j];
    }
}

template < typename T, typename F >
void mergesort(T* a, unsigned int size, std::size_t left, std::size_t right, F f)
{

    if (right - left > 0)
    {
        ::mergesort(a, size, left, left + (right - left) / 2, f);
        ::mergesort(a, size, left + (right - left) / 2 + 1, right, f);

        ::merge(a, size, left, right, f);
    }
    else
    {
        return;
    }
}

template <typename T, typename F>
void sort(T* a, unsigned int size, F f)
{
    ::mergesort(a, size, 0, size - 1, f);
}


template <typename T, typename F, std::size_t N>
void sort(T (&a) [N], F f)
{
    ::mergesort(&a[0], N, 0, N - 1, f);
}

In [ ]:
//first task .cpp file

#include <iostream>
#include <vector>
#include "header1.h"


int main()
{
    const unsigned int size = 5U;
    //a dynamic array
    int* a = new int[size]{ 1, 500, -2, 3, 0 };

    //sort by ascending order
    ::sort(a, size, [](auto i, auto j) { return i < j; });

    for (auto i = 0U; i < size; i++)
        std::cout << a[i] << ' ';
    

    std::cout << std::endl;

    //an array
    int b[size] = { 3, 2, 4, 7, -100 };

    //sort by descending order
    ::sort(b, [](auto i, auto j) { return i > j; });

    for (auto i = 0U; i < size; i++)
        std::cout << b[i] << ' ';

    delete[] a;
    return 0;
}


##                                                                           Task 2

In [ ]:
#include <algorithm>
#include <cassert>
#include <iostream>
#include <vector>
#include <xstring>
#include <string>


class VectorException : public std::exception
{
private:
    std::string m_error;
public:
	 VectorException(const char * error) :
		std::exception(error), m_error(error) {}

	explicit VectorException(const std::string& error) :
		std::exception(error.c_str()), m_error(error) {}


	~VectorException() noexcept = default;

	const char* what() const noexcept override
	{
		return m_error.c_str();
	}
};



template < typename T >
class Vector
{
public:

	using pointer = T*;
	using const_pointer = const T*;
	using reference = T&;
	using const_reference = const T&;
	using size_type = std::size_t;

	using iterator = pointer;
	using const_iterator = const_pointer;

public:
	Vector() noexcept = default;

	Vector(T i) noexcept  : m_size(1), m_capacity(1)
	{
		m_data = new T[m_size];
		m_data[0] = i;
	}

	Vector(const Vector& other) noexcept;
	Vector(Vector&& other) noexcept;

	Vector& operator=(const Vector& other) noexcept;
	Vector& operator=(Vector&& other) noexcept;

	void swap(Vector& other) noexcept;

	void print() const
	{
		for (auto i = 0U; i < m_size; ++i)
			std::cout << m_data[i] << ' ';
		std::cout << std::endl;
	}


	~Vector() noexcept
	{
		delete[] m_data;
	}

public:

	reference operator[] (unsigned int index)
	{
		if (index >= m_size)
			throw VectorException("Index out of range");
		return m_data[index];
	}
	const_reference operator[](unsigned int index) const
	{
		if (index >= m_size)
			throw VectorException("Index out of range");
		return m_data[index];
	}

	iterator begin() noexcept
	{
		return m_data;
	}
	iterator end() noexcept
	{
		return m_data + m_size; 
	}

	const_iterator begin() const noexcept
	{
		return m_data;
	}
	const_iterator end() const noexcept
	{
		return m_data + m_size;
	}

	size_type size() const noexcept
	{
		return m_size;
	}

	size_type capacity() const noexcept
	{
		return m_capacity;
	}

	void resize(unsigned int n, const T& value);
	void resize(unsigned int n);

	void push_back(const T& value) noexcept;
	void pop_back()
	{
		if (m_size == 0)
			throw VectorException("Index out of range");
		m_size--;
	}


private:

	pointer m_data{ nullptr };
	size_type m_size{ 0 };
	size_type m_capacity{ 0 };
};



template < typename T >
Vector < T >::Vector(const Vector& other) noexcept :
	m_size{ other.m_size }, m_capacity(other.capacity)
{
	m_data = new T[m_size];
	for (auto i = 0; i < m_size; ++i)
		m_data[i] = other[i];
}

template < typename T >
Vector < T >::Vector(Vector&& other) noexcept :
	m_size(other.m_size), m_data(other.m_data), m_capacity(other.capacity)
{
	other.m_size = 0;
	other.m_data = nullptr;
	other.capacity = 0;
}	


template < typename T >
Vector < T >& Vector < T > ::operator= (const Vector& other) noexcept
{
	if (this == &other) return *this;
	if (m_data)
	{
		delete[] m_data;
	}
	m_size = other.m_size;
	m_capacity = other.capacity;
	m_data = new T[m_size];
	for (auto i = 0; i < m_size; ++i)
		m_data[i] = other.m_data[i];
	return *this;
}

template < typename T >
Vector < T >& Vector < T > ::operator= (Vector<T>&& other) noexcept
{
	if (this == &other)
		return *this;
	if (m_data)
	{
		delete[] m_data;
	}
	m_size = other.m_size;
	m_data = other.m_data;
	m_capacity = other.capacity;
	other.m_m_data = nullptr;
	other.m_size = 0;
	other.capacity = 0;
	return *this;
}


template < typename T >
void Vector<T>::push_back(const T& value) noexcept
{
	if (m_size >= m_capacity)
	{
		auto new_capacity = m_capacity == 0 ? 1 : 2 * m_capacity;
		auto new_data = new T[new_capacity];
		std::copy(this->begin(), this->end(), new_data);
		delete[] m_data;
		m_data = new_data;
		m_capacity = new_capacity;
	}
	m_data[m_size++] = value;
}

template <typename T>
void Vector<T>::resize(unsigned int new_size, const T & value)
{
	if (new_size > m_size)
	{
		if (new_size > m_capacity)
		{
			auto new_capacity = new_size;
			auto new_data = new T[new_capacity];
			std::copy(this->begin(), this->end(), new_data);
			for (auto i = m_size; i < new_size; ++i)
				new_data[i] = value;
			delete[] m_data;
			m_data = new_data;
			m_capacity = new_capacity;
		}
		else
		{
			for (auto i = m_size; i < new_size; ++i)
				m_data[i] = value;
			m_size = new_size;
		}
	}
	else
		m_size = new_size;
}

template <typename T>
void Vector<T>::resize(unsigned int new_size)
{
	if (new_size > m_size)
		throw VectorException("No value to fill");
	if (new_size > m_capacity)
	{
		m_capacity = new_size;
	}
	m_size = new_size;
}

template < typename T >
void Vector<T>::swap(Vector& other) noexcept
{
	std::swap(m_size, other.m_size);
	std::swap(m_data, other.m_data);
	std::swap(m_capacity, other.m_capacity);
}


template <>
class Vector < std::string>
{
private:
	std::string* m_data;
	std::size_t m_size;

public:
	Vector(const std::string& s) :
		m_size(std::size(s))
	{
		m_data = new std::string[m_size];
		for (auto i = 0U; i < m_size; ++i)
			m_data[i] = s[i];
	}

	~Vector()
	{
		delete[] m_data;
	}

	void print() const
	{
		for (auto i = 0U; i < m_size; ++i)
			std::cout << m_data[i] << ' ';
		std::cout << std::endl;
	}
};



int main()
{
	try
	{
		std::string s = "sdsfdf";
		Vector <std::string> v(s);
		v.print();

		Vector <int> v1(8);
		v1.push_back(36);
		v1.push_back(23);
		v1.push_back(3);
		v1.print();
		v1.pop_back();
		v1.print();

		v1.resize(1, 5);
		v1.print();

		Vector <char> c('a');
		c.push_back('b');
		c.resize(9, 'c');
		c.print();
		c.resize(2);
		c.print();


		std::cout << v1[9];
	}
	catch (const VectorException& e)
	{
		std::cerr << e.what() << std::endl;
	}

	system("pause");

	return EXIT_SUCCESS;
}

# 8-ая неделя:

### Контрольные вопросы:
1) Какие существуют способы обработки различных ошибок?
    Обработка “на месте”:
    1. if-else (+ while) [проверка пользовательского ввода]
        1.1. “Способ реагирования”: [аварийное завершение работы]
    abort (деструктор)  /terminate (исключения, noexcept) /exit
    2. assert [проверка утверждений] - макрос препроцессора, который обрабатывает условное выражение во время выполнения.(пример ошибки: разыменование nullptr)
    Обработка “не на месте”:
    1. Механизм исключений (C++ - style)
    2. Коды возврата (C - style) [Например: оператор возврата return]
    3. Другие способы

2) В чем заключаются недостатки механизма кодов возврата?
    а) Возвращаемые значения не всегда понятны. Если функция, выполняющая бинарный поиск, возвращает -1, то непонятно - это код ошибки или найденное значение.
    б) Функции могут возвращать только одно значение. Если нужно будет вернуть и результат выполнения функции, и код возврата, придётся придумывать что-то ещё, например, использовать дополнительную переменную-флаг. 
    в) Не сработает для конструктора и различных операторов.

3) Какими особенностями обладает механизм исключений?
    1) Многоуровневость: f() -> g() -> h()  исключение, возникшее в h(), будет поймано в f()
    2) Эффективность: если нет исключения, то нет побочных расходов.
    3) Преобразования: переход от производного класса к базовому (Derived -> Base)

4) Для чего используется спецификатор и оператор noexcept?
    -спецификатор noexcept - спецификатор времени компиляции, используется для того, чтобы функция не выбрасывала исключения самостоятельно.
    -оператор noexcept - возвращает true/false. Может использоваться внутри функции.  Данный оператор удобно использовать для задания условия выполнения кода: является ли переданный фрагмент кода потенциально выбрасывающим исключения или нет.

5) Как формулируются гарантии безопасности исключений?
    + Базовая (Например, идиома RAII)
        - инварианты не нарушены
        - нет утечек ресурсов
    + Строгая
        - обеспечивает транзакционное поведение
        - Отсутствие исключений

##                                                                        Task 1

In [ ]:
#include <iostream>
#include <vector>
#include <cassert>
#include <variant>


enum class outcome
{
	success,
	invalid_index,
	out_of_range
};

std::variant<outcome, int>  getValue(const std::vector<int>& a, int i)
{
	if (i < 0 || i >= std::size(a))
		return outcome::invalid_index;
	return a[i];
}

outcome printVector(const std::vector<int>& a, unsigned int n)
{
	if (n >= std::size(a))
		return outcome::out_of_range;
	for (auto i = 0U; i < n; i++)
	{
		std::cout << a[i] << " ";
	}
	std::cout << std::endl;
	return outcome::success;
}

bool isError(outcome result)
{
	switch (result)
	{
	case outcome::invalid_index:
	case outcome::out_of_range:
		std::cout << "error" << std::endl;
		return true;
	case outcome::success:
		std::cout << "completed successfully" << std::endl;
		return false;
	default: 
		return false;
	}
}



int main()
{
	std::vector<int> a{ 1, 2, 3 };

	size_t n;

	//std::cin >> n;

	n = 2;

	if (isError(printVector(a, n)))
	{
		return EXIT_FAILURE;
	}


	std::variant<outcome, int> res = getValue(a, 5);

	if (std::holds_alternative<outcome>(res) && isError(std::get<outcome>(res)))
	{	
			return EXIT_FAILURE;
	}
	else
	{
		std::cout << std::get<int>(res) << std::endl;
	}


	system("pause");
	return EXIT_SUCCESS;
}

##                                                                       Task 2

In [ ]:
#include <iostream>
#include <vector>
#include <cassert>
#include <exception>
#include <stdexcept>

class Vector
{
private:
	std::vector<int> m_v;

public:
	Vector(const std::vector<int>& v = {}) noexcept : m_v(v) {};

	Vector(size_t i)
	{
		for (auto j = 0U; j < i; ++j)
			m_v.push_back(0);
	};

	~Vector() noexcept = default;

	int operator[](int i) const;

	Vector sqrt();

	int size() const;
};

void length_limit(size_t i)
{
	if (i >= 3)
		throw std::length_error("the limit length has been exceeded");
}

int Vector::operator[](int i) const
{
	if (i < 0 || std::size(m_v) <= i)
		throw std::out_of_range{ "vector index out of range" };
	return m_v[i];
}

int Vector::size() const
{
	return std::size(m_v);
}

Vector Vector::sqrt()
{
	Vector tmp(std::size(m_v));
	for (auto i = 0U; i < std::size(m_v); ++i)
	{
		if (m_v[i] < 0)
			throw std::domain_error{ "not defined" };
		tmp.m_v[i] = (std::sqrt(m_v[i]));
	}
	*this = tmp;
	return *this;
}

double division(double x, double y)
{
	if (y == 0)
		throw(std::runtime_error("division by zero"));
	return x / y;
}

void comp(int x, int y)
{
	if (x = y)
		throw std::logic_error("invalid structure");
}

int main()
{
	Vector v_1;
	Vector v_2({ 0, -1, 4 });


	try
	{
		std::cout << v_1[3];
	}
	catch(const std::out_of_range& e)
	{
		std::cerr << e.what() << std::endl;
	}
	catch (...) //catch the rest
	{
		std::cerr << "unknown ex\n";
	}


	try
	{
		v_2.sqrt();
		for (auto i = 0U; i < v_2.size(); i++)
			std::cout << v_2[i] << std::endl;
	}
	catch(const std::domain_error& e)
	{
		std::cerr << e.what() << std::endl;
	}
	catch (...) 
	{
		std::cerr << "unknown ex\n";
	}


	try
	{
		int n = v_2.size();
		length_limit(n);
	}
	catch (const std::length_error& e)
	{
		std::cerr << e.what() << std::endl;
	}
	catch (...) 
	{
		std::cerr << "unknown ex\n";
	}


	try
	{
		division(3.0, 0.0);
	}
	catch (const std::runtime_error& e)
	{
		std::cerr << e.what() << std::endl;
	}
	catch (...)
	{
		std::cerr << "unknown ex\n";
	}


	try
	{
		comp(2, 3);
	}
	catch (const std::logic_error& e)
	{
		std::cerr << e.what() << std::endl;
	}
	catch (...)
	{
		std::cerr << "unknown ex\n";
	}


	return 0;
}

##                                                                 Task 3

In [ ]:
#header file

#pragma once
#include <iostream>
#include <string>

namespace Algebra
{
	class FractionException : public std::exception
	{
	private:
		std::string m_error;
	public:
		FractionException(const std::string& error) noexcept :
			std::exception(error.c_str()), m_error(error) {}


		~FractionException() noexcept = default;

		const char* what() const noexcept override
		{
			return m_error.c_str();
		}
	};

	class Fraction
	{
	public:

		Fraction(int numerator = 0, int deminator = 1) :
			m_numerator(numerator), m_denominator(deminator) {
			if (m_denominator == 0)
				throw FractionException("devision by zero");
			reduction();
		}

		~Fraction() noexcept = default;

		void reduction() noexcept;

		friend Fraction operator+(const Fraction& f1, const Fraction& f2);
		Fraction& operator+=(const Fraction& f1);

		friend Fraction operator-(const Fraction& f1, const Fraction& f2);
		Fraction& operator-=(const Fraction& f1);

		friend Fraction operator*(const Fraction& f1, const Fraction& f2);
		Fraction& operator*=(const Fraction& f1);

		friend Fraction operator/(const Fraction& f1, const Fraction& f2);
		Fraction& operator/=(const Fraction& f1);



		friend bool operator==(const Fraction& f1, const Fraction& f2) noexcept;

		friend bool operator<=(const Fraction& f1, const Fraction& f2) noexcept;

		friend bool operator>=(const Fraction& f1, const Fraction& f2) noexcept;

		friend bool operator!=(const Fraction& f1, const Fraction& f2) noexcept;

		friend bool operator>(const Fraction& f1, const Fraction& f2) noexcept;

		friend bool operator<(const Fraction& f1, const Fraction& f2) noexcept;

		friend std::ostream& operator<<(std::ostream& out, const Fraction& f);
		friend std::istream& operator>>(std::istream& in, Fraction& f);

		Fraction& operator++() noexcept;
		Fraction& operator--() noexcept;
		Fraction operator++(int) noexcept;
		Fraction operator--(int) noexcept;

		explicit operator double() const noexcept;

		auto get_numerator() const noexcept
		{
			return m_numerator;
		}

		auto get_denominator() const noexcept
		{
			return m_denominator;
		}

	private:
		int m_numerator;
		int m_denominator;

	};

}

In [ ]:
#func cpp

#include <iostream>
#include "header.hpp"

namespace Algebra
{
	int gcd(int a, int b)
	{
		return (b == 0) ? (a > 0 ? a : -a) : gcd(b, a % b);
	}


	void Fraction::reduction() noexcept
	{
		if (m_denominator < 0)
		{
			m_numerator *= (-1);
			m_denominator *= (-1);
		}
		int gcd_ = gcd(m_denominator, m_numerator);
		m_numerator /= gcd_;
		m_denominator /= gcd_;
	}


	Fraction& Fraction::operator+=(const Fraction& f1)
	{
		*this = Fraction(m_numerator * f1.m_denominator + f1.m_numerator * m_denominator, m_denominator * f1.m_denominator);
		return *this;
	}

	Fraction operator+(const Fraction& f1, const Fraction& f2)
	{
		Fraction result(f1);
		result += f2;
		return result;
	}


	Fraction operator-(const Fraction& f1, const Fraction& f2)
	{
		Fraction result(f1);
		result -= f2;
		return result;
	}

	Fraction& Fraction::operator-=(const Fraction& f1)
	{
		*this = Fraction(m_numerator * f1.m_denominator - f1.m_numerator * m_denominator, m_denominator * f1.m_denominator);
		return *this;
	}

	Fraction operator*(const Fraction& f1, const Fraction& f2)
	{
		Fraction result(f1);
		result *= f2;
		return result;
	}

	Fraction& Fraction::operator*=(const Fraction& f1)
	{
		*this = Fraction(m_numerator * f1.m_numerator, m_denominator * f1.m_denominator);
		return *this;
	}

	Fraction operator/(const Fraction& f1, const Fraction& f2)
	{
		Fraction result(f1);
		result /= f2;
		return result;
	}

	Fraction& Fraction::operator/=(const Fraction& f1)
	{
		if (f1.m_numerator == 0)
			throw FractionException("devision by zero");
		*this = Fraction(m_numerator * f1.m_denominator, m_denominator * f1.m_numerator);
		return *this;
	}

	bool operator==(const Fraction& f1, const Fraction& f2) noexcept
	{
		return (!(f1 > f2) && !(f1 < f2));
	}


	bool operator!=(const Fraction& f1, const Fraction& f2) noexcept
	{
		return (f1 < f2 || f1 > f2);
	}


	bool operator>=(const Fraction& f1, const Fraction& f2) noexcept
	{
		return (f1 > f2 || (!(f1 > f2) && !(f1 < f2)));
	}


	bool operator<=(const Fraction& f1, const Fraction& f2) noexcept
	{
		return (f1 < f2 || (!(f1 > f2) && !(f1 < f2)));
	}


	bool operator>(const Fraction& f1, const Fraction& f2) noexcept
	{
		return (f1.m_numerator * f2.m_denominator > f2.m_numerator * f1.m_denominator);
	}

	bool operator<(const Fraction& f1, const Fraction& f2) noexcept
	{
		return (f1.m_numerator * f2.m_denominator < f2.m_numerator* f1.m_denominator);
	}


	std::ostream& operator<<(std::ostream& out, const Fraction& f)
	{
		if (f.m_denominator == 1)
		{
			out << f.m_numerator;
		}
		else
		{
			out << f.m_numerator << "/" << f.m_denominator;
		}
		return out;
	}

	std::istream& operator>>(std::istream& in, Fraction& f)
	{
		char c;
		in >> f.m_numerator;
		in >> c;
		in >> f.m_denominator;

		if (f.m_denominator == 0)
			throw FractionException("devision by zero");

		f.reduction();
		return in;
	}

	Fraction& Fraction::operator++() noexcept
	{
		m_numerator += m_denominator;
		reduction();
		return *this;
	}

	Fraction& Fraction::operator--() noexcept
	{
		m_numerator -= m_denominator;
		reduction();
		return *this;
	}

	Fraction Fraction::operator++(int) noexcept
	{
		Fraction temp(m_numerator, m_denominator);
		++(*this);
		reduction();
		return temp;
	}

	Fraction Fraction::operator--(int) noexcept
	{
		Fraction temp(m_numerator, m_denominator);
		--(*this);
		reduction();
		return temp;
	}

	Fraction::operator double() const noexcept
	{
		return static_cast<double>(m_numerator) / m_denominator;
	}
}

In [ ]:
#main cpp

#include <iostream>
#include "header.hpp"


int main()
{
	try
	{
		//using exception
		Algebra::Fraction f(-2, 0);

		Algebra::Fraction n(6);
		Algebra::Fraction y;

		Algebra::Fraction x1(3);
		Algebra::Fraction x2(2);

		//using exception
		x1 / 0;

		//using exception
		std::cout << "x1 = " << x1 << std::endl;
		std::cout << "x2 = " << x2 << std::endl;

		std::cout << "(x1 < x2) = " << (x1 < x2) << std::endl;
		std::cout << "(x1 > x2) = " << (x1 > x2) << std::endl;
		std::cout << "(x1 == x2) = " << (x1 == x2) << std::endl;
		std::cout << "(x1 != x2) = " << (x1 != x2) << std::endl;
		std::cout << "(x1 <= x2) = " << (x1 <= x2) << std::endl;
		std::cout << "(x1 >= x2) = " << (x1 >= x2) << std::endl;

		std::cout << "n = " << n << std::endl;
		std::cout << "++n = " << ++n << std::endl;
		std::cout << "n++ = " << n++ << std::endl;
		std::cout << "n = " << n << std::endl;
		std::cout << "--n = " << --n << std::endl;
		std::cout << "n-- = " << n-- << std::endl;
		std::cout << "n = " << n << std::endl;

		std::cout << "f = " << f << std::endl;
		std::cout << "f + n = " << f + n << std::endl;
		std::cout << "f - n = " << f - n << std::endl;
		std::cout << "f * n = " << f * n << std::endl;
		std::cout << "f / n = " << f / n << std::endl;

		std::cout << "y = ";

		std::cin >> y;

		std::cout << "f += y\nf = " << (f += y) << std::endl;
		std::cout << "f -= y\nf = " << (f += y) << std::endl;
		std::cout << "f *= y\nf = " << (f += y) << std::endl;
		std::cout << "f /= y\nf = " << (f += y) << std::endl;
		std::cout << "f = " << f << std::endl;

		Algebra::Fraction h(-3);
		Algebra::Fraction l(-3, 2);

		std::cout << l / h;
	}
	catch(const Algebra::FractionException & e)
	{
		std::cerr << e.what() << std::endl;
	}



	return 0;
}

##                                                                   Task 4
    Пути нормального выполнения.
1. Если (e.title() == "CEO") истинно (e.salary() даже не будет вычисляться, но вывод в std::cout будет.
2. Если (e.title() != "CEO") и (e.salary() > 100000) истинно, вычислены корректно обе части, присутствует вывод в std::cout.
3. Если (e.title() != "CEO") и (e.salary() <= 100000), вычислены корректно обе части, спустимся в else, осуществляется вывод в std::cout.
    
    Пути с исключениями.
1. String evaluate_salary_and_return_name (Employee e)
Вызывается конструктор копирования Employee, так как аргумент передается по значению, где также может генерироваться исключения.
2. e.title() может генерировать исключение.
3. Из-за оператора == необходимо сделать приведение типа "CEO", может быть исключение при конструировании.
4. Оператор == может представлять собой функцию, генерирующую исключения.
5. e.salary() может генерировать исключение.
6. > может также потребовать приведение типа, создание временного объекта для 100000.
7. Оператор > может представлять собой пользовательскую функцию, генерирующую исключения.
8. Оператор ||  может представлять собой пользовательскую функцию, генерирующую исключения.  
9, 10, 11, 12. std::cout << e.name() << " " << e.surname() << " is overpaid.\n";
    Оператор << может генерировать исключения. У нас таких точек четыре.
13. e.name() может генерировать исключения.
14. e.surname() может генерировать исключения.
15, 16. std::cout << e.name() << " is overpaid.\n";
    Оператор << может генерировать исключения. У нас таких точек две.
17. e.name() может генерировать исключения.
18. return e.name() + " " + e.surname();
    e.name() может генерировать исключения.
19. e.surname() может генерировать исключения.
20. При сложении может понадобиться создание временного объекта, приведение типа для оператора + с " ".
21, 22. Оператор + может представлять собой пользовательскую функцию, генерирующую исключения.

    Итого: 25 путей.

# 6-ая неделя:

### Контрольные вопросы:
1) Перечислите все специальные функции-члены класса, включая перемещающие операции.
    T() - Конструктор по умолчанию
    T(...) - User-defined конструктор
    ~T() - Деструктор
    T & operator = (const T&)
    T & operator = (T&&)
    T(const T&) - copy конструктор
    T(T &&) - move конструктор 

2) Приведите примеры операторов, которые можно, нельзя и не рекомендуется перегружать.
    Можно: ->  []  ()
    Нельзя:  ::  .  .*
    Не рекомендуется: &  ,
 
3) О каких преобразованиях следует помнить при проектировании операторов?
    Следует помнить о неявных преобразованиях типов.

4) Опишите классификацию выражений на основе перемещаемости и идентифицируемости.
    lvalue (i && !m) - объект, занимаемый идентифицируемое место в памяти, можно получить его адрес; идентифицируемый и неперемещаемый;
    prvalue (!i && m) - не идентифицируемый и перемещаемый;
    xvalue (i && m) - идентифицируемый и перемещаемый;
    glvalue (i) - идентифицируемый; 
    rvalue (m) - неадресуемый объект в памяти; перемещаемый.

5) Зачем нужны rvalue-ссылки?
    Rvalue-ссылки нужны для продления времени жизни временных объектов, т.е. для реализации семантики перемещения.

6) Почему семантика перемещения лучше копирования?
    Перемещение позволяет не использовать дополнительную память, как это делается при копировании. Как следствие - повышение производительности.

7) Что делает функция std::move и когда нет необходимости явно ее вызывать?
    Функция std::move не перемещает объект, она выполняет приведение типа.
    Т.е. она возвращает rvalue-ссылку, поэтому можем использовать семантику перемещения. Std::move можно явно не вызывать при возвращении значения функцией, потому что компилятор сам выполняет return value optimization (RVO). Явный вызов функции в данном случае будет излишним.

8) Кем выполняется непосредственная работа по перемещению?
    Перемещающий оператор присваивания/перемещающий конструктор выполняют непосредственную работу по перемещению.

9) Когда может потребоваться пользовательская реализация специальных функций-членов класса?
	а) Если нужно внести какие-то пользовательские правки в специальные функции-члены.
	б) Если необходима оптимизация реализации функций-членов по умолчанию.

10) Для чего нужны ключевые слова default и delete в объявлении специальных функций-членов класса?
    а) default - указание для компилятора сгенерировать конструктор/деструктор по умолчанию;
    б) delete - указание для компилятора не вызывать/определять функции-члены, к которым оно применимо.


##                                                                      Task 1

In [ ]:
// header file

#pragma once
#include <iostream>
#include <vector>

class Array
{
private:
	std::size_t m_length;
	int* m_array = new int[m_length];
	std::vector<int> m_vector;

public:

	Array();

	explicit Array(int i);

	Array(const Array& other);

	Array(Array&& other);

	auto getter() const
	{
		for (auto i = 0; i < m_length; ++i)
		{
			std::cout << m_array[i] << " ";
		}
		std::cout << std::endl;
	}

	Array& operator= (const Array& other);

	Array& operator= (Array&& other);

	~Array();

};

In [ ]:
// func file

#include "header.h"
#include <iostream>
#include <vector>


Array::Array() : m_array(nullptr), m_length(0), m_vector({})
{}

Array::Array(int i) : m_length(1), m_vector({})
{
	m_array = new int[m_length];
	m_array[0] = i;
	m_vector.push_back(i);
}


Array::Array(const Array& other) : m_array(nullptr), m_length(other.m_length), m_vector(other.m_vector)
{
	if (other.m_array)
	{
		m_array = new int[m_length];
		for (auto i = 0; i < m_length; ++i)
		{
			m_array[i] = other.m_array[i];
		}
	}
}

Array::Array(Array&& other) :
	m_length(other.m_length), m_array(other.m_array), m_vector(std::move(other.m_vector))
{
	other.m_length = 0;
	other.m_array = nullptr;
	other.m_vector.clear();
}

Array::~Array()
{
	if (m_array)
	{
		delete[] m_array;
		m_vector.clear();
	}
}


Array& Array::operator= (const Array& other)
{
	if (m_vector == other.m_vector)
		return *this;
	if (m_array)
	{
		delete[] m_array;
	}
	m_length = other.m_length;
	m_vector = other.m_vector;
	m_array = new int[m_length];
	for (auto i = 0; i < m_length; ++i)
	{
		m_array[i] = other.m_array[i];
	}
	return *this;
}

Array& Array::operator= (Array&& other)
{
	if (m_vector == other.m_vector)
		return *this;
	if (m_array)
	{
		delete[] m_array;
		m_vector.clear();
	}
	m_length = other.m_length;
	m_array = other.m_array;
	m_vector = other.m_vector;
	other.m_array = nullptr;
	other.m_vector.clear();
	other.m_length = 0;
	return *this;
}

In [ ]:
// main file

#include <iostream>
#include <vector>
#include "header.h"


Array generateItem(int x)
{
	Array item(x);
	return item;
}

int main()
{
	Array a(4);
	Array b(a);
	Array c(generateItem(42));
	Array g;


	a = b;
	g = generateItem(31);
	
	return 0;
}

##                                                                           Task 2

In [ ]:
// header file

#pragma once
#include <iostream>

namespace NS
{
	class Fraction
	{
	public:

		Fraction(int numerator = 0, int deminator = 1) :
			m_numerator(numerator), m_denominator(deminator) {
			reduction();
		}

		Fraction(const Fraction& a) : m_numerator(a.m_numerator), m_denominator(a.m_denominator) { reduction(); }

		~Fraction() = default;

		void reduction();

		friend Fraction operator+(const Fraction& f1, const Fraction& f2);
		Fraction& operator+=(const Fraction& f1);

		friend Fraction operator-(const Fraction& f1, const Fraction& f2);
		Fraction& operator-=(const Fraction& f1);

		friend Fraction operator*(const Fraction& f1, const Fraction& f2);
		Fraction& operator*=(const Fraction& f1);

		friend Fraction operator/(const Fraction& f1, const Fraction& f2);
		Fraction& operator/=(const Fraction& f1);



		friend bool operator==(const Fraction& f1, const Fraction& f2);

		friend bool operator<=(const Fraction& f1, const Fraction& f2);

		friend bool operator>=(const Fraction& f1, const Fraction& f2);

		friend bool operator!=(const Fraction& f1, const Fraction& f2);

		friend bool operator>(const Fraction& f1, const Fraction& f2);

		friend bool operator<(const Fraction& f1, const Fraction& f2);

		friend std::ostream& operator<<(std::ostream& out, const Fraction& f);
		friend std::istream& operator>>(std::istream& in, Fraction& f);

		Fraction& operator++();
		Fraction& operator--();
		Fraction operator++(int);
		Fraction operator--(int);

		explicit operator double() const;

		auto get_numerator() const
		{
			return m_numerator;
		}

		auto get_denominator() const
		{
			return m_denominator;
		}

	private:
		int m_numerator;
		int m_denominator;

	};
}

In [ ]:
// func file

#include <iostream>
#include "header.hpp"

namespace NS
{
	int gcd(int a, int b)
	{
		return (b == 0) ? (a > 0 ? a : -a) : gcd(b, a % b);
	}


	void Fraction::reduction()
	{
		if (m_denominator < 0)
		{
			m_numerator *= (-1);
			m_denominator *= (-1);
		}
		int gcd_ = gcd(m_denominator, m_numerator);
		m_numerator /= gcd_;
		m_denominator /= gcd_;
	}


	Fraction& Fraction::operator+=(const Fraction& f1)
	{
		*this = Fraction(m_numerator * f1.m_denominator + f1.m_numerator * m_denominator, m_denominator * f1.m_denominator);
		return *this;
	}

	Fraction operator+(const Fraction& f1, const Fraction& f2)
	{
		Fraction result(f1);
		result += f2;
		return result;
	}


	Fraction operator-(const Fraction& f1, const Fraction& f2)
	{
		Fraction result(f1);
		result -= f2;
		return result;
	}

	Fraction& Fraction::operator-=(const Fraction& f1)
	{
		*this = Fraction(m_numerator * f1.m_denominator - f1.m_numerator * m_denominator, m_denominator * f1.m_denominator);
		return *this;
	}

	Fraction operator*(const Fraction& f1, const Fraction& f2)
	{
		Fraction result(f1);
		result *= f2;
		return result;
	}

	Fraction& Fraction::operator*=(const Fraction& f1)
	{
		*this = Fraction(m_numerator * f1.m_numerator, m_denominator * f1.m_denominator);
		return *this;
	}

	Fraction operator/(const Fraction& f1, const Fraction& f2)
	{
		Fraction result(f1);
		result /= f2;
		return result;
	}

	Fraction& Fraction::operator/=(const Fraction& f1)
	{
		*this = Fraction(m_numerator * f1.m_denominator, m_denominator * f1.m_numerator);
		return *this;
	}

	bool operator==(const Fraction& f1, const Fraction& f2)
	{
		return (!(f1 > f2) && !(f1 < f2));
	}


	bool operator!=(const Fraction& f1, const Fraction& f2)
	{
		return (f1 < f2 || f1 > f2);
	}


	bool operator>=(const Fraction& f1, const Fraction& f2)
	{
		return (f1 > f2 || (!(f1 > f2) && !(f1 < f2)));
	}


	bool operator<=(const Fraction& f1, const Fraction& f2)
	{
		return (f1 < f2 || (!(f1 > f2) && !(f1 < f2)));
	}


	bool operator>(const Fraction& f1, const Fraction& f2)
	{
		return (f1.m_numerator * f2.m_denominator > f2.m_numerator * f1.m_denominator);
	}

	bool operator<(const Fraction& f1, const Fraction& f2)
	{
		return (f1.m_numerator * f2.m_denominator < f2.m_numerator* f1.m_denominator);
	}


	std::ostream& operator<<(std::ostream& out, const Fraction& f)
	{
		if (f.m_denominator == 1)
		{
			out << f.m_numerator;
		}
		else
		{
			out << f.m_numerator << "/" << f.m_denominator;
		}
		return out;
	}

	std::istream& operator>>(std::istream& in, Fraction& f)
	{
		char c;
		in >> f.m_numerator;
		in >> c;
		in >> f.m_denominator;

		f.reduction();
		return in;
	}

	Fraction& Fraction::operator++()
	{
		m_numerator += m_denominator;
		reduction();
		return *this;
	}

	Fraction& Fraction::operator--()
	{
		m_numerator -= m_denominator;
		reduction();
		return *this;
	}

	Fraction Fraction::operator++(int)
	{
		Fraction temp(m_numerator, m_denominator);
		++(*this);
		reduction();
		return temp;
	}

	Fraction Fraction::operator--(int)
	{
		Fraction temp(m_numerator, m_denominator);
		--(*this);
		reduction();
		return temp;
	}

	Fraction::operator double() const
	{
		return static_cast<double>(m_numerator) / m_denominator;
	}
}

In [ ]:
// main file

#include <iostream>
#include "header.hpp"


int main()
{
	NS::Fraction f(-2, 3);
	NS::Fraction n(6);
	NS::Fraction y;

	NS::Fraction x1(3);
	NS::Fraction x2(2);

	std::cout << "x1 = " << x1 << std::endl;
	std::cout << "x2 = " << x2 << std::endl;

	std::cout << "(x1 < x2) = " << (x1 < x2) << std::endl;
	std::cout << "(x1 > x2) = " << (x1 > x2) << std::endl;
	std::cout << "(x1 == x2) = " << (x1 == x2) << std::endl;
	std::cout << "(x1 != x2) = " << (x1 != x2) << std::endl;
	std::cout << "(x1 <= x2) = " << (x1 <= x2) << std::endl;
	std::cout << "(x1 >= x2) = " << (x1 >= x2) << std::endl;

	std::cout << "n = " << n << std::endl;
	std::cout << "++n = " << ++n << std::endl;
	std::cout << "n++ = " << n++ << std::endl;
	std::cout << "n = " << n << std::endl;
	std::cout << "--n = " << --n << std::endl;
	std::cout << "n-- = " << n-- << std::endl;
	std::cout << "n = " << n << std::endl;

	std::cout << "f = " << f << std::endl;
	std::cout << "f + n = " << f + n << std::endl;
	std::cout << "f - n = " << f - n << std::endl;
	std::cout << "f * n = " << f * n << std::endl;
	std::cout << "f / n = " << f / n << std::endl;

	std::cout << "y = ";

	std::cin >> y;

	std::cout << "f += y\nf = " << (f += y) << std::endl;
	std::cout << "f -= y\nf = " << (f += y) << std::endl;
	std::cout << "f *= y\nf = " << (f += y) << std::endl;
	std::cout << "f /= y\nf = " << (f += y) << std::endl;
	std::cout << "f = " << f << std::endl;

	NS::Fraction h(-3);
	NS::Fraction l(-3, 2);

	std::cout << l / h;


	return 0;
}

# 5-ая неделя:

### Контрольные вопросы:
1) На чём основано объектно-ориентированное программирование?
- Инкапсуляция: приватные данные, публичные методы (функции-члены)  
- Полиморфизм: одинаковый интерфейс, разные реализации  
- Наследование: потомки (производные классы) наследуют свойства родителей (базовые классы)  
		Дополнительно: абстракция.

2) Какие аспекты стоит учитывать при проектировании классов?
    а) Необходимо указывать секции: private/public/protected.
    б) Располагать данные-члены и функции-члены следует в таком порядке:
    конструкторы/деструкторы → другие спец. функции-члены → геттеры/сеттеры → данные-члены.
    в) Маленькие функции-члены стоит располагать внутри класса (inline), а большие вне класса (например, объявление класса в .hpp файле, а функции-члены в отдельном .cpp)
    + ещё какие-то  

3) Почему удобно разделять классы на интерфейс и реализацию?
    При разделении класса на интерфейс (.hpp файл) и реализацию (.cpp файл) удаётся скрыть способ осуществления работоспособности класса.
    Также если класс используется в нескольких файлах проекта удобно вынести его объявление в отдельный файл.
    !Акцент на модульности и ускорении процесса компиляции!

4) Чем внутреннее связывание отличается от внешнего связывания?
    а) Внешнее связывание: обратиться к сущности, неопределенной в данном юните трансляции (например, определим эту сущность в файле header.hpp, а затем напишем #include “header.cpp” в файле task.cpp, т.е. там, где хотим воспользоваться сущностью). Внешним связыванием пользуемся мы и линковщик.
    б) Внутреннее связывание: например, static int x = 6.  
    Если захотим обратиться к данной переменной в другом юните трансляции, то необходимо будет написать в нём extern int x; - это уже пример внешнего связывания. 
    Внутреннее связывание отличается от внешнего тем, что к сущности имеющей внутреннее связывание нельзя обратиться из единицы трансляции, отличной от той, где она определена.

5) Какими особенностями обладают именованные пространства имён?
 - Свойство аддитивности:

In [ ]:
namespace NS1
{
  void f()
  {}
}

namespace NS1
{
  void g()
  {}
}

Итого в NS1 будет и void f() и void g().

- Свойство вложенности:

In [ ]:
namespace NS1
{
  namespace NS2
  {
    void g()
    {}
  }
}

Возможность ссылаться на внешнее имя с помощью псевдонимов.
Данные свойства могут быть удобны в использовании в больших проектах.

##                                                                          Task 1

In [ ]:
// header file

#pragma once

#include <iostream>
#include <limits>


class F
{

public:

	F(): 
		m_coef_1(3.0), m_coef_2(4.0), m_coef_3(2.0)
	{}


	F(double a, double b, double c) :
		m_coef_1(a), m_coef_2(b), m_coef_3(c)
	{}

	void print_equ() const;
	bool is_on_line(double x = 0.0, double y = -0.5) const;

	~F()
	{
		std::cout << "Destructing a line\n";
	}

private:

	double m_coef_1;
	double m_coef_2;
	double m_coef_3;
};

In [ ]:
// func file

#include <iostream>
#include <limits>
#include "header.hpp"


bool F::is_on_line(double x, double y) const
{
	constexpr double epsilon = std::numeric_limits < double > ::epsilon();

	std::cout << "The point coordinates : (" << x << ", " << y << ")\n";

	if (m_coef_1 * x + m_coef_2 * y + m_coef_3 < epsilon)
	{
		return true;
	}
	else
	{
		return false;
	}
}

void F::print_equ() const
{
	std::cout << "The linear equation is " << m_coef_1 << "x + " << m_coef_2 << "y + " << m_coef_3 <<
		" = 0\n";
}

In [ ]:
// main file

#include <iostream>
#include "header.hpp"

int main()
{
	F f(2.0, 1.0, 5.0);
	f.print_equ();

	double x = 2.0;
	double y = 5.0;

	//std::cout << "Enter the point coordinates: ";
	//std::cin >> x;
	//std::cin >> y;

	std::cout << std::boolalpha << f.is_on_line(x, y) << std::endl;

	F g;
	g.print_equ();

	std::cout << std::boolalpha << g.is_on_line() << std::endl;

	system("pause");

	return 0;
}

##                                                                           Task 2

In [ ]:
#include <iostream>

class Energy
{
public:

	Energy(double value) :
		m_value(value)
	{}

	auto getter() const
	{
		return m_value;
	}

	auto setter_erg()
	{
		m_value *= joule_in_erg;
	}


	auto setter_eV()
	{
		m_value *= joule_in_ev;
	}

	~Energy()
	{}

	static constexpr double joule_in_erg = 1e8;
	static constexpr double joule_in_ev = 6.241509e18;

private:

	double m_value;
};

enum class units
{
	erg,
	eV
};

int main()
{
	double v;

	std::cout << "Converter from joule to erg and electronvolt.\nEnter the value: ";
	std::cin >> v;

	Energy e(v);

	std::cout << "Choose units to convert:\n0 - erg;\n1 - electronvolt.\nEnter a number of the option: ";

	int type;
	units t;

	std::cin >> type;

	t = static_cast <units>(type);

	switch (t)
	{
	case units::erg:
		e.setter_erg();
		break;
	case units::eV:
		e.setter_eV();
		break;
	default:
		break;
	}

	std::cout << e.getter() << std::endl;

	system("pause");

	return 0;
}

##                                                                            Task 3

In [ ]:
#include <iostream>

class A;

class B
{
public:

    B(double salary, const std::string& name) :
        m_salary(salary), m_name(name)
    {}

    class Key {
        friend class A;
        Key() {};
    };

    auto getter(Key) const { return m_salary; }

    ~B() {}

private:
    double m_salary;
    std::string m_name;
};

class A
{
private:
    B::Key k;

public:

    auto get_salary(B b) { return b.getter(k); }
};



int main()
{
    B b(42.9, "Kate");
    A a;
    std::cout << a.get_salary(b);

	return 0;
}

##                                                                            Task 4

In [ ]:
// first example: .hpp
#pragma once
#include <string>

class Person
{
public:

	Person(const std::string& name, int x) :
		m_name(name), m_x(x)
	{}
	// Присутствует ссылка на неразрешенный внешний символ. Код компилируется, потому 
	//что видит объявление функции, однако линковщик при попытке разрешить все внешние связи выдаст ошибку,
	//так как функция нигде не определена. 
	void f();

private:

	std::string m_name;
	int m_x;
};

In [ ]:
// first example: .cpp
#include "header1.hpp"
#include <iostream>
#include <string>

int main()
{
	Person p("name", 42);

	p.f(); 

	system("pause");

	return 0;
}

In [ ]:
// second example: .hpp
#pragma once
#include <string>

//Обнаружен многократно определенный символ. Происходит это, потому что заголовочные файлы 
//отдельно компилятором не компилируются. Они включаются в cpp файл и после этого компилятор это компилирует 
//и получает цельный main.cpp. Здесь все хорошо. Потом та же процедура проделывается с другим срр - файлом
//func.cpp Компилятор, опять, вместо #include "header2.hpp" вставит содержимое файла "header2.hpp". 
//И откомпилирует, поскольку компиляция независимая и получит файл func.cpp. Таким образом в обоих срр- файлах
//будут определены int f(int m) и i. Линковщик при компоновке выдаст ошибку, так как одна и та же переменная 
//и функция были определены дважды. Исправить эту проблему достаточно легко: для этого в заголовочном файле 
//нужно писать только заголовки функции и extern ссылки на переменные, определенные вне. Определять все лучше
//в отдельном срр файле, которым, например, у нас является func.cpp.

int i = 5;

int f(int m) {
    return m * 2;
}

In [ ]:
// second example: func.сpp

#include "header2.hpp"
#include <iostream>

In [ ]:
// second example: main.сpp

#include "header2.hpp"
#include <iostream>
#include <string>

int main()
{

	system("pause");

	return 0;
}

In [ ]:
// header file


In [ ]:
// func file

In [ ]:
// main file